# Proyecto de Análisis de Datos de Lichess

Primero, la base de datos de Lichess está disponible [aquí](https://database.lichess.org/). Descargué el archivo con los juegos "estándar" de Noviembre del 2023 con extensión .pgn.zst, el cual extraje usando PeaZip.

Creé una tabla en MySQL:

In [ ]:
CREATE TABLE games_metadata(
    id INT PRIMARY KEY AUTO_INCREMENT,
    gameType VARCHAR(255),
    white VARCHAR(255),
    black VARCHAR(255),
    result VARCHAR(255),
    utcDate DATE,
    utcTime TIME,
    whiteElo INT,
    blackElo INT
)

Después usé el siguiente código de python para guardar los metadatos relevantes (nombres de los jugadores, su rating, el resultado, el control de tiempo y fecha) de cada juego en la tabla de MySQL previamente creada:

In [ ]:
import mysql.connector
from chess.pgn import read_game

path_to_pgn_data = "database/lichess_db_standard_rated_2023-11.pgn"
insert_into_database_sql_string = '''
        INSERT INTO games_metadata (gameType, white, black, result, utcDate, utcTime, whiteElo, blackElo) 
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
        '''
connect_params = {
    "host":"",
    "user":"",
    "password":"",
    "database":""} # Este diccionario debe contener la información relevante para que el conector de MySQL se conecte a la base de datos

def connect_to_db_and_upload_list_of_rows(list_of_tuples: list[tuple]):
    db = mysql.connector.connect(**connect_params)

    cursor = db.cursor()
    cursor.executemany(
        insert_into_database_sql_string, 
        list_of_tuples)
    db.commit()
    db.close()

# Para no exceder el RAM, inserto en bloques de 10,000 filas. Si esto se hace en múltiples sesiones hay que usar el parámetro empty_iterations
def insert_into_database(data_path: str = path_to_pgn_data, chunksize: int = 10_000, iterations: int = 100, empty_iterations: int = 0):
    with open(data_path) as file:
        for _ in range(empty_iterations):
            game = read_game(file)
        for i in range(iterations):
            list_of_rows = []
            for _ in range(chunksize):
                game = read_game(file)
                data_tuple = (game.headers["Event"].split(" ")[1], 
                            game.headers["White"],
                            game.headers["Black"], 
                            game.headers["Result"],
                            game.headers["UTCDate"].replace(".", "-"),
                            game.headers["UTCTime"],
                            game.headers["WhiteElo"],
                            game.headers["BlackElo"])

                list_of_rows.append(data_tuple)

            connect_to_db_and_upload_list_of_rows(list_of_rows)

Primero quería un dataset lo más grande posible, pero debido a cuellos de botella (en especial con la función "read_pgn"), elegí usar solo los juegos de un día (el código previo guarda los juegos en orden): el primero de Noviembre.

Usando la extensión SQLTools de vscode y el siguiente código creé archivos csv para cada control de tiempo, incluyendo una columna para todos los distintos jugadores, una columna para el número de juegos con blancas, una columna para los juegos con negras, y otra para el total:

In [ ]:
SELECT
    w.player AS player,
    w.num_games_as_white AS white_games,
    b.num_games_as_black AS black_games,
    w.num_games_as_white + b.num_games_as_black AS total
FROM
    (SELECT
        DISTINCT white AS player,
        COUNT(*) AS num_games_as_white
    FROM
        games_metadata
    WHERE
        utcDate = "2023-11-01" AND gameType = "Correspondence" -- La segunda condición puede ser alterada para enfocarse en un control de tiempo específico, o puede ser eliminada para tomar todos los controles de tiempo simultaneamente
    GROUP BY
        white
    ORDER BY
        num_games_as_white DESC) AS w
    INNER JOIN
    (SELECT
        DISTINCT black AS player,
        COUNT(*) AS num_games_as_black
    FROM
        games_metadata
    WHERE
        utcDate = "2023-11-01" AND gameType = "Correspondence" -- Lo mismo que el comentario previo
    GROUP BY
        black
    ORDER BY
        num_games_as_black DESC) AS b
    ON w.player = b.player
ORDER BY
    total DESC,
    player

Para referencias futuras nombré los archivos csv "MostActivePlayersAll.csv", "MostActivePlayersClassical.csv", etc. Esto será importante para la gráfica de barras de Pareto.

También copié la tabla, primero a csv, y luego a .pickle con los tipos de dato correctos, y lo llamé "NovemberFirst" para usarlo facilmente con pandas.

Para la información de la gráfica de pastel usé el siguiente código:

In [ ]:
from pandas import DataFrame, Series, read_pickle, read_csv

df: DataFrame = read_pickle("database/NovemberFirst.pickle")

total_games = len(df.index)
total_games_correspondence = len(df[df["gameType"] == "Correspondence"].index)
total_games_classical = len(df[df["gameType"] == "Classical"].index)
total_games_rapid = len(df[df["gameType"] == "Rapid"].index)
total_games_blitz = len(df[df["gameType"] == "Blitz"].index)
total_games_bullet = len(df[df["gameType"] == "Bullet"].index)
total_games_ultrabullet = len(df[df["gameType"] == "UltraBullet"].index)

print(f"There where a total of {total_games}\n")
print(f"{total_games_correspondence} correspondence games, equivalent to {round(total_games_correspondence*100/total_games, 2)}%")
print(f"{total_games_classical} classical games, equivalent to {round(total_games_classical*100/total_games, 2)}%")
print(f"{total_games_rapid} rapid games, equivalent to {round(total_games_rapid*100/total_games, 2)}%")
print(f"{total_games_blitz} blitz games, equivalent to {round(total_games_blitz*100/total_games, 2)}%")
print(f"{total_games_bullet} bullet games, equivalent to {round(total_games_bullet*100/total_games, 2)}%")
print(f"{total_games_ultrabullet} ultra bullet games, equivalent to {round(total_games_ultrabullet*100/total_games, 2)}%")

Para el gráfico de Pareto necesitamos saber cuántos jugadores distintos hay en cada control de tiempo:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

df = df[df["gameType"] == "Blitz"] # Esta línea puede ser alterada para distintos controles de tiempo, o eliminada para tomar todos los juegos simultaneamente
players = []
players.extend(list(df["white"]))
players.extend(list(df["black"]))
distinct_players = list(set(players))

print(f"There where {len(distinct_players)} distinct players")

Ahora podemos adquirir las cifras para el gráfico de Pareto:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

game_type = "Blitz" # This line and the next can be altered to focus on different time controls, or removed to get the full picture
df = df[df["gameType"] == game_type]
players = list(read_csv("database/MostActivePlayersBlitz.csv")["player"]) # this line should be altered like the previous one
total_players = 377134 # Here we write the number of total distinct players for the corresponding game type
total_games = 1481528 # Here we write the number of total games for the corresponding game type
for i in [5, 10, 20, 33, 50]:
    number = len(df[(df["white"].isin(players[:int(total_players*(i/100))])) | (df["black"].isin(players[:int(total_players*(i/100))]))])
    print(f"The {i}% most active {game_type.lower()} players are involved in {number} games, which is {round(number*100/total_games, 2)}% of the total")

Finalmente, para la distribución usé lo siguiente:

In [ ]:
df: DataFrame = read_pickle("database/NovemberFirst.pickle")

df["avgRating"] = (df["whiteElo"] + df["blackElo"]) / 2
df["avgRatingByIncrements"] = df["avgRating"] - (df["avgRating"] % 50)
df["avgRatingByIncrements"] = df["avgRatingByIncrements"].astype(int)

n_by_avgRating: Series = df.groupby("avgRatingByIncrements")["white"].count()
print(n_by_avgRating)